# Introduction

In RAI, the Configuration is a central concept representing the complete state of the robotic environment, encompassing both the robot and any objects in the world. It serves as a container for all relevant simulation information, including the positions, orientations, and kinematic relationships of objects and robot links.

Another key term is Frame. A Frame represents a reference point or a rigid body in the environment. It defines the position, orientation, and kinematic relationships of objects or robot components. Frames can have shapes (such as boxes or spheres) attached to them and are connected by joints or constraints, forming a kinematic tree.

In [1]:
import robotic as ry

In [2]:
# Create our configuration 
C = ry.Config()

# Add the first frame.
f = C.addFrame(name="first")
f.setShape(type=ry.ST.marker, size=[.3])
f.setPosition([0.0 ,0.0 , 0.5])
f.setQuaternion([1.0, 0.3, 0.0, .0]) 
print("frame name:", f.name)
print("pos:", f.getPosition()) 
print("quat:", f.getQuaternion())
C.view()


frame name: first
pos: [0.  0.  0.5]
quat: [0.95782629 0.28734789 0.         0.        ]


0

C.view() opens a view window. You can right-click on the window bar and select "Always on Top."

# Adding a Box to the Environment

Let's clear the configuration and add a new box object.

In [3]:
C.clear()

In [4]:
C.addFrame("box1") \
    .setPosition([0, 0, .25]) \
    .setShape(ry.ST.ssBox, size=[.5, .5, .5, .05]) \
    .setColor([1, .5, 0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

# Relative Positions

Add a second box 1 meter above the first one.

In [5]:
box2 = C.addFrame(name="box2", parent="box1")
box2.setShape(ry.ST.ssBox, size=[.5, .5, .5, .05])
box2.setRelativePosition([0,0,1])
box2.setColor([0,0,1])

C.view()

0

Display the position and orientation of the second box.

In [6]:
f = C.getFrame("box2")
print("position:", f.getPosition()) 
print("orientation:", f.getQuaternion())

position: [0.   0.   1.25]
orientation: [1. 0. 0. 0.]


# Creating Two Link Manipulator

We will create a two-link planar manipulator system using .g files. The two_link_manipulator.g file defines the links and joints of the manipulator. We simply need to use the ry.addFile() method to add the contents of the .g files to the configuration. Additionally, we will include the Franka Panda robot using these files.

In [7]:
K = ry.Config()
K.addFile("two_link_manipulator.g")
K.view()

0

# Features

In [8]:
C.clear()

C = ry.Config()
C.addFile(ry.raiPath("panda/panda.g"))
q = C.getJointState()
[y,J] = C.eval(ry.FS.position, ["gripper"])
print("feature value:", y, "\nJacobian:", J)
C.view()

feature value: [2.81896272e-01 1.25455202e-16 8.10370138e-01] 
Jacobian: [[-1.25455202e-16  4.77370138e-01 -1.15172364e-16 -2.37213253e-01
   0.00000000e+00  3.96841887e-02  0.00000000e+00  0.00000000e+00]
 [ 2.81896272e-01  6.25935464e-17  5.54002326e-01  2.79346636e-16
   1.54786187e-01  2.18260648e-16  4.16333634e-17  0.00000000e+00]
 [ 0.00000000e+00 -2.81896272e-01 -1.79370329e-16  5.03226163e-01
   0.00000000e+00  2.24676245e-01  0.00000000e+00  0.00000000e+00]]


0

In [9]:
del C
del K

# Inverse Kinematics

Add Franka Panda robot to the configuration

In [1]:
import robotic as ry
import numpy as np
import time
C = ry.Config()
C.addFile(ry.raiPath("../rai-robotModels/scenarios/pandaSingle.g"))
C.view()

0

Add the target frame

In [11]:
target = C.addFrame("target", "table")
target.setShape(ry.ST.marker, [.1])
target.setRelativePosition([0., .3, .3])
pos = target.getPosition()
cen = pos.copy()
C.view()

0

A basic inverse kinematics approach

In [12]:
def IK(C, target, qHome):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos
    , [1e-1], qHome)
    komo.addObjective([], ry.FS.positionDiff, ["l_gripper", target]
    , ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    return [komo.getPath()[0], ret]

In [13]:
qHome = C.getJointState()
q_target, ret = IK(C, "target", qHome)
print(ret)
C.setJointState(q_target)
C.view()

{ time: 0.000559, evals: 6, done: 1, feasible: 1, sos: 0.00676232, f: 0, ineq: 0, eq: 0.00135438 }


0

Now let’s do it in the loop for better understanding

In [14]:
for t in range(20):
    time.sleep(.1)
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)
    q_target, ret = IK(C, "target", qHome)
    print(ret)
    C.setJointState(q_target)
    C.view()

{ time: 0.000298, evals: 3, done: 1, feasible: 1, sos: 0.00390117, f: 0, ineq: 0, eq: 0.0022241 }
{ time: 0.000394, evals: 3, done: 1, feasible: 1, sos: 0.00355451, f: 0, ineq: 0, eq: 0.000849066 }
{ time: 0.000362, evals: 3, done: 1, feasible: 1, sos: 0.00381623, f: 0, ineq: 0, eq: 0.00236149 }
{ time: 0.00068, evals: 4, done: 1, feasible: 1, sos: 0.00362223, f: 0, ineq: 0, eq: 0.000198029 }
{ time: 0.000501, evals: 3, done: 1, feasible: 1, sos: 0.00383193, f: 0, ineq: 0, eq: 0.00259145 }
{ time: 0.000334, evals: 3, done: 1, feasible: 1, sos: 0.00335681, f: 0, ineq: 0, eq: 0.0021803 }
{ time: 0.000346, evals: 4, done: 1, feasible: 1, sos: 0.00340078, f: 0, ineq: 0, eq: 0.000200279 }
{ time: 0.00048, evals: 4, done: 1, feasible: 1, sos: 0.00394477, f: 0, ineq: 0, eq: 0.000380498 }
{ time: 0.000315, evals: 3, done: 1, feasible: 1, sos: 0.00395594, f: 0, ineq: 0, eq: 0.00191533 }
{ time: 0.000346, evals: 4, done: 1, feasible: 1, sos: 0.00427722, f: 0, ineq: 0, eq: 0.000228223 }
{ time: 0

# Theory of Inverse Kinematics

Now, we will solve IK numerically from scratch. Here, we will use the theory behind it, and we will help our robot reach its goal step by step. For this, we will return to our two-link manipulator.

In [6]:
import robotic as ry
import numpy as np
K = ry.Config()
K.addFile("two_link_manipulator.g")
K.view()

0

And here is the basic implementation of theoretical inverse kinematics.

In [8]:
n = K.getJointDimension()
q = K.getJointState()
w = 1e-1
W = w * np.identity(n)
y_target = [0.0, 0.7, 0.7]

for i in range(10):
    y, J = K.eval(ry.FS.position, ["end-effector"])
    q += np.linalg.inv(J.T @ J + W) @ J.T @ (y_target - y)
    K.setJointState(q)
    K.view()